In [1]:
import os
count=0;
import cv2

In [2]:
for filename in os.listdir('C:/Users/Vicky singh/Desktop/fight-detection-surv-dataset/fight'):
    vidObj = cv2.VideoCapture('C:/Users/Vicky singh/Desktop/fight-detection-surv-dataset/fight/'+filename) 
   
    while True:
        success,image = vidObj.read() 
        if success:
            image=cv2.resize(image,(240,240))
            cv2.imwrite("C:/Users/Vicky singh/Desktop/fight-detection-surv-dataset/fight1/frame%d.jpg" % count, image)
            cv2.imshow("image", image)
            count += 1
        else:
            break
        vidObj.release() 
        cv2.destroyAllWindows() 

In [3]:
count=0

In [4]:
for filename in os.listdir('C:/Users/Vicky singh/Desktop/fight-detection-surv-dataset/noFight'):
    vidObj = cv2.VideoCapture('C:/Users/Vicky singh/Desktop/fight-detection-surv-dataset/noFight/'+filename) 
   
    while True:
        success,image = vidObj.read() 
        if success:
            image=cv2.resize(image,(240,240))
            cv2.imwrite("C:/Users/Vicky singh/Desktop/fight-detection-surv-dataset/nofight1/frame%d.jpg" % count, image)
            cv2.imshow("image", image)
            count += 1
        else:
            break
        vidObj.release() 
        cv2.destroyAllWindows() 

In [5]:
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
#from data import DataSet
import os.path

In [6]:
def get_model(weights='imagenet'):
    # create the base pre-trained model
    base_model = InceptionV3(weights=weights, include_top=False)

    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    # and a logistic layer
    predictions = Dense(2, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [7]:
def get_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        horizontal_flip=True,
        rotation_range=10.,
        width_shift_range=0.2,
        height_shift_range=0.2)

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        'C:/Users/Vicky singh/Desktop/fight-detection-surv-dataset/Dataset',
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical')

    return train_generator

In [8]:
model = get_model()
generators = get_generators()

Found 8177 images belonging to 2 classes.


In [9]:
checkpointer = ModelCheckpoint(
    filepath=os.path.join('data', 'checkpoints', 'inception.{epoch:03d}.hdf5'),
    verbose=1,
    save_best_only=True)

# Helper: Stop when we stop learning.
early_stopper = EarlyStopping(patience=10)

# Helper: TensorBoard
tensorboard = TensorBoard(log_dir=os.path.join('data', 'logs'))

In [10]:
def train_model(model, nb_epoch, generators, callbacks=[]):
    train_generator =generators
    model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=nb_epoch,
        callbacks=callbacks)
    return model

In [11]:
def freeze_all_but_top(model):
    """Used to train just the top layers of the model."""
    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
    for layer in model.layers[:-2]:
        layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def freeze_all_but_mid_and_top(model):
    """After we fine-tune the dense layers, train deeper."""
    # we chose to train the top 2 inception blocks, i.e. we will freeze
    # the first 172 layers and unfreeze the rest:
    for layer in model.layers[:172]:
        layer.trainable = False
    for layer in model.layers[172:]:
        layer.trainable = True

    # we need to recompile the model for these modifications to take effect
    # we use SGD with a low learning rate
    model.compile(
        optimizer=SGD(lr=0.0001, momentum=0.9),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'top_k_categorical_accuracy'])

    return model

In [ ]:
model = freeze_all_but_top(model)
model = train_model(model, 10, generators)

C:\Users\Vicky singh\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
100/100 [==============================] - 562s 6s/step - loss: 0.0179 - accuracy: 1.0000
Epoch 2/10
 68/100 [===================>..........] - ETA: 2:44 - loss: 0.0000e+00 - accuracy: 1.0000

In [ ]:
  for file in os.listdir('C:/Users/Vicky singh/Desktop/fight-detection-surv-dataset/Dataset'):
        print('C:/Users/Vicky singh/Desktop/fight-detection-surv-dataset/Dataset/'+file)

In [ ]:
import shutil 

location="C:/Users/Vicky singh/Desktop/fight-detection-surv-dataset"
path = os.path.join(location, '.ipynb_checkpoints') 
print(path)
# removing directory 
shutil.rmtree(path) 

In [ ]:
for file in os.listdir('C:/Users/Vicky singh/Desktop/fight-detection-surv-dataset/Dataset'):
    print('C:/Users/Vicky singh/Desktop/fight-detection-surv-dataset/Dataset/'+file)

In [ ]:
model = freeze_all_but_mid_and_top(model)
model = train_model(model, 10, generators,
                        [checkpointer, early_stopper, tensorboard])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(model.history.history['loss'])

In [ ]:
  plt.plot(model.history.history['accuracy'])

In [ ]:

#for filename in os.listdir('/content/fight-detection-surv-dataset/noFight'):
  #print(filename)
vidObj = cv2.VideoCapture('C:/Users/Vicky singh/Desktop/fight-detection-surv-dataset/test/fi008.mp4') 
while True: 
        success, image = vidObj.read() 
        #print(success)
        # Saves the frames with frame-count 
        if success:
            image=cv2.resize(image,(299,299))
            cv2.imwrite("C:/Users/Vicky singh/Desktop/fight-detection-surv-dataset/frame%d.jpg" % count, image) 
            count += 1
        else:
            break
        vidObj.release() 
        cv2.destroyAllWindows() 

In [ ]:
import numpy as np

In [ ]:
for image1 in os.listdir('C:/Users/Vicky singh/Desktop/fight-detection-surv-dataset/test/test_frame'):
    image = cv2.imread(os.path.join('C:/Users/Vicky singh/Desktop/fight-detection-surv-dataset/test/test_frame',image1))
    if image is not None:
        x = np.expand_dims(image, axis=0)
        images = np.vstack([x])
        classes = model.predict(images, batch_size=10)
        print(classes[0])
        if classes[0][0]>0.5:
            print("Fighting recorded")
        else:
            print("No fight")
            
  

In [ ]:
validation_generator